In [ ]:
###pip install chromedriver-autoinstaller####

import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller  # Import chromedriver_autoinstaller

# Instal and config auto ChromeDriver
chromedriver_autoinstaller.install()

driver = webdriver.Chrome()
driver.get("https://iprsearch.ipindia.gov.in/publicsearch")

# Wait until the element with ID "TextField7" is visible on the web page
wait = WebDriverWait(driver, 10)
input_field_1 = wait.until(EC.visibility_of_element_located((By.ID, "TextField7")))
input_field_2 = wait.until(EC.visibility_of_element_located((By.ID, "FromDate")))
input_field_3 = wait.until(EC.visibility_of_element_located((By.ID, "ToDate")))

# Enter values in the input fields
input_field_2.clear()
input_field_2.send_keys("03/01/2005")
input_field_3.clear()
input_field_3.send_keys("01/01/2010")
input_field_1.clear()
input_field_1.send_keys("INDIA")

# Scroll to the bottom of the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Zoom in on the left part of the page
driver.execute_script("document.querySelector('.container').style.width='200%'")

alphanumeric_text = input("Enter Captcha: ")

# Wait for the text input field to be visible
text_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "CaptchaText"))
)

# Fill in the text input field with the captcha value
text_input.send_keys(alphanumeric_text)

# Find the "Search" button element and click it
submit_button = driver.find_element(By.CSS_SELECTOR, "input[name='submit'][value='Search']")
submit_button.click()

# Open the CSV file in write mode
filename = 'data.csv'
with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the CSV header
    header = ['Invention Title', 'Publication Number', 'Publication Date', 'Publication Type', 'Application Number',
              'Application Filing Date', 'Priority Number', 'Priority Country', 'Priority Date', 'Field Of Invention',
              'Classification (IPC)', 'Applicant Name', 'Applicant Address', 'Applicant Country', 'Applicant Nationality',
              'Inventor Name', 'Inventor Address', 'Inventor Country', 'Inventor Nationality']
    writer.writerow(header)

    # Loop through all result pages
    while True:
        # Find all buttons with the name "ApplicationNumber"
        buttons = driver.find_elements(By.NAME, "ApplicationNumber")

        # For each button, click it to open the link in a new tab
        for button in buttons:
            button.click()
            driver.switch_to.window(driver.window_handles[-1])  # Switch to the new tab

            # Process the new tab
            title = driver.title
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            data = {}
            table = soup.find('table', class_='table-striped')
            rows = table.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 2:
                    key = cells[0].text.strip()
                    value = cells[1].text.strip()
                    data[key] = value

            # Extract applicant and inventor details
            

            inventor_table = soup.find('td', text='Inventor').find_next('table')
            inventor_rows = inventor_table.find_all('tr')[1:]  # Exclude the header row

            inventors = []
            for row in inventor_rows:
                name, address, country, nationality = row.find_all('td')
                inventor = {
                    'Name': name.text.strip(),
                    'Address': address.text.strip(),
                    'Country': country.text.strip(),
                    'Nationality': nationality.text.strip()
                }
                inventors.append(inventor)
                
            applicant_table = soup.find('td', text='Applicant').find_next('table')
            applicant_rows = applicant_table.find_all('tr')[1:]  # Exclude the header row

            applicants = []
            for row in applicant_rows:
                name, address, country, nationality = row.find_all('td')
                applicant = {
                    'Name': name.text.strip(),
                    'Address': address.text.strip(),
                    'Country': country.text.strip(),
                    'Nationality': nationality.text.strip()
                }
            applicants.append(applicant)

            # Write the data to a CSV row
            row_data = [
                data.get('Invention Title', ''), data.get('Publication Number', ''), data.get('Publication Date', ''),
                data.get('Publication Type', ''), data.get(' Application Number', ''), data.get('Application Filing Date', ''),
                data.get('Priority Number', ''), data.get('Priority Country', ''), data.get('Priority Date', ''),
                data.get('Field Of Invention', ''), data.get('Classification (IPC)', ''),
                ', '.join([a['Name'] for a in applicants]),
                ', '.join([a['Address'] for a in applicants]),
                ', '.join([a['Country'] for a in applicants]),
                ', '.join([a['Nationality'] for a in applicants]),
                ', '.join([i['Name'] for i in inventors]),
                ', '.join([i['Address'] for i in inventors]),
                ', '.join([i['Country'] for i in inventors]),
                ', '.join([i['Nationality'] for i in inventors])
            ]
            writer.writerow(row_data)

            try:
                driver.close()
            except:
                pass
            driver.switch_to.window(driver.window_handles[0])

        # Find the "Next" button if available
        next_button = None
        try:
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "next")))
        except:
            break

        # Click the "Next" button to go to the next page
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        driver.execute_script("arguments[0].click();", next_button)

# Close the Chrome driver
driver.quit()